In [1]:
import os
import sys
parent_dir = os.path.abspath('../../..')
sys.path.append(parent_dir)
import pandas as pd
import numpy as np
from eval_utils import compare_MLE, data_profiling, cate_info, col_check,  transform_label
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
import os
import json
import pickle
import torch
import random

In [2]:
import tabpfn
from tabpfn_extensions import unsupervised
from tabpfn_extensions import TabPFNClassifier, TabPFNRegressor
from tabpfn_extensions.many_class import ManyClassClassifier

# Adult

In [16]:
path = "../../sample/Adult/"
df25 = pd.read_csv(path + "df25.csv")
df50 = pd.read_csv(path + "df50.csv")
df100 = pd.read_csv(path + "data100.csv")
df200 = pd.read_csv(path + "data200.csv")
df400 = pd.read_csv(path + "data400.csv")
df800 = pd.read_csv(path + "data800.csv")

In [9]:
data_lst = [df25, df50]
seed_lst = list(range(5))
x_cols = ['age', 'workclass', 'education', 'education-num',
          'marital-status', 'occupation', 'relationship', 'race', 'sex',
          'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']
y_col = "Income"
cate_var = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'Income']
bool_var = []
cols = x_cols + [y_col]
num_var = list((set(cols) - set(cate_var)) - set(bool_var))


res = {}
res_shape = {}

cate_cols = []
for var in cate_var:
    cate_cols.append(cols.index(var))

In [12]:
ns = [25, 50]
for i in range(2):
    data_len = ns[i]
    data = data_lst[i]
    test_data = pd.read_csv(path + f"df{data_len}_test.csv")
    data = data[cols]
    data_comb = pd.concat([data, test_data])
    data_comb.reset_index(inplace=True, drop=True)
    encoder_dict = {}
    for col in cate_var:
        encoder = LabelEncoder()
        print(col)
        encoder.fit(data_comb[col])
        data[col] = encoder.transform(data[col])
        encoder_dict[col] = encoder
    X = torch.tensor(data.to_numpy())
    clf = TabPFNClassifier()
    reg = TabPFNRegressor()
    model = unsupervised.TabPFNUnsupervisedModel(
        tabpfn_clf = clf, tabpfn_reg = reg
    )

    model.set_categorical_features(cate_cols)
    model.fit(X)
    for j in range(5):
        torch.manual_seed(j)
        torch.cuda.manual_seed(j)
        torch.cuda.manual_seed_all(j)
        np.random.seed(j)
        random.seed(j)
        syn = model.generate_synthetic_data(n_samples=data_len)
        syn_df = pd.DataFrame(syn.numpy(), columns=cols)
        for k, encoder in encoder_dict.items():
            syn_df[k] = syn_df[k].astype(int)
            syn_df[k] = encoder.inverse_transform(syn_df[k])
        syn_df.to_csv(f"gen/Adult/{data_len}/TabPFN/df_{j}.csv", index=False)
            

workclass
education
marital-status
occupation
relationship
race
sex
native-country
Income


  0%|          | 0/14 [00:00<?, ?it/s]

0


/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:227: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_sampled = pred["criterion"].sample(torch.tensor(pred["logits"]), t=t)
/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:166: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(d["logits"]) for d in densities],
  7%|▋         | 1/14 [00:00<00:05,  2.34it/s]

1


...

/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:227: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_sampled = pred["criterion"].sample(torch.tensor(pred["logits"]), t=t)
/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:166: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(d["logits"]) for d in densities],
 86%|████████▌ | 12/14 [00:09<00:02,  1.17s/it]

12
12
12


 93%|█████████▎| 13/14 [00:10<00:01,  1.01s/it]

13
13
13


100%|██████████| 14/14 [00:10<00:00,  1.28it/s]


In [17]:
data_lst = [df100, df200, df400, df800]
seed_lst = list(range(5))
x_cols = ['age', 'workclass', 'education', 'education-num',
          'marital-status', 'occupation', 'relationship', 'race', 'sex',
          'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']
y_col = "Income"
cate_var = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'Income']
bool_var = []
cols = x_cols + [y_col]
num_var = list((set(cols) - set(cate_var)) - set(bool_var))


res = {}
res_shape = {}

cate_cols = []
for var in cate_var:
    cate_cols.append(cols.index(var))

In [19]:
ns = [100, 200, 400, 800]
for i in range(4):
    data_len = ns[i]
    data = data_lst[i]
    test_data = pd.read_csv(path + f"data_test.csv")
    data = data[cols]
    data_comb = pd.concat([data, test_data])
    data_comb.reset_index(inplace=True, drop=True)
    encoder_dict = {}
    for col in cate_var:
        encoder = LabelEncoder()
        print(col)
        encoder.fit(data_comb[col])
        data[col] = encoder.transform(data[col])
        encoder_dict[col] = encoder
    X = torch.tensor(data.to_numpy())
    clf = TabPFNClassifier()
    reg = TabPFNRegressor()
    model = unsupervised.TabPFNUnsupervisedModel(
        tabpfn_clf = clf, tabpfn_reg = reg
    )

    model.set_categorical_features(cate_cols)
    model.fit(X)

    path_tabpfn = "gen/Adult/{data_len}/TabPFN/"
    os.makedirs(path_tabpfn, exist_ok=True)
    for j in range(5):
        torch.manual_seed(j)
        torch.cuda.manual_seed(j)
        torch.cuda.manual_seed_all(j)
        np.random.seed(j)
        random.seed(j)
        syn = model.generate_synthetic_data(n_samples=data_len)
        syn_df = pd.DataFrame(syn.numpy(), columns=cols)
        for k, encoder in encoder_dict.items():
            syn_df[k] = syn_df[k].astype(int)
            syn_df[k] = encoder.inverse_transform(syn_df[k])
        syn_df.to_csv(f"gen/Adult/{data_len}/TabPFN/df_{j}.csv", index=False)

workclass
education
marital-status
occupation
relationship
race
sex
native-country
Income


  0%|          | 0/14 [00:00<?, ?it/s]

0


/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:227: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_sampled = pred["criterion"].sample(torch.tensor(pred["logits"]), t=t)
/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:166: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(d["logits"]) for d in densities],
  7%|▋         | 1/14 [00:00<00:08,  1.49it/s]

1


 14%|█▍        | 2/14 [00:01<00:06,  1.85it/s]


2


ValueError: Number of classes 14 exceeds the maximal number of classes supported by TabPFN. Consider using a strategy to reduce the number of classes. For code see https://github.com/PriorLabs/tabpfn-extensions/blob/main/src/tabpfn_extensions/many_class/many_class_classifier.py

# Insurance

In [18]:
path = "../../sample/Insurance/"
df25 = pd.read_csv(path + "df25.csv")
df50 = pd.read_csv(path + "df50.csv")

data_lst = [df25, df50]

In [19]:
seed_lst = list(range(5))

x_cols = ['age', 'sex', 'bmi', 'children', 'smoker', 'region']
y_col = 'charges'
bool_var = []
cols = x_cols + [y_col]
cate_var = ['sex', 'smoker', 'region']
num_var = list(set(cols) - set(cate_var))

res_shape = {}
res = {}
cate_cols = []
for var in cate_var:
    cate_cols.append(cols.index(var))


In [20]:
ns = [25, 50]
for i in range(2):
    data_len = ns[i]
    data = data_lst[i]
    test_data = pd.read_csv(path + f"df{data_len}_test.csv")
    data = data[cols]
    data_comb = pd.concat([data, test_data])
    data_comb.reset_index(inplace=True, drop=True)
    encoder_dict = {}
    for col in cate_var:
        encoder = LabelEncoder()
        print(col)
        encoder.fit(data_comb[col])
        data[col] = encoder.transform(data[col])
        encoder_dict[col] = encoder
    X = torch.tensor(data.to_numpy())
    clf = TabPFNClassifier()
    reg = TabPFNRegressor()
    model = unsupervised.TabPFNUnsupervisedModel(
        tabpfn_clf = clf, tabpfn_reg = reg
    )

    model.set_categorical_features(cate_cols)
    model.fit(X)
    for j in range(5):
        torch.manual_seed(j)
        torch.cuda.manual_seed(j)
        torch.cuda.manual_seed_all(j)
        np.random.seed(j)
        random.seed(j)
        syn = model.generate_synthetic_data(n_samples=data_len)
        syn_df = pd.DataFrame(syn.numpy(), columns=cols)
        for k, encoder in encoder_dict.items():
            syn_df[k] = syn_df[k].astype(int)
            syn_df[k] = encoder.inverse_transform(syn_df[k])
        syn_df.to_csv(f"gen/Insurance/{data_len}/TabPFN/df_{j}.csv", index=False)

sex
smoker
region


  0%|          | 0/7 [00:00<?, ?it/s]

0


/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:227: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_sampled = pred["criterion"].sample(torch.tensor(pred["logits"]), t=t)
/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:166: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(d["logits"]) for d in densities],
 29%|██▊       | 2/7 [00:00<00:01,  3.40it/s]

1
2


...

/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:227: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_sampled = pred["criterion"].sample(torch.tensor(pred["logits"]), t=t)


6


/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:227: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_sampled = pred["criterion"].sample(torch.tensor(pred["logits"]), t=t)


6


/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:227: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_sampled = pred["criterion"].sample(torch.tensor(pred["logits"]), t=t)
/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:166: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(d["logits"]) for d in densities],
100%|██████████| 7/7 [00:05<00:00,  1.31it/s]


# ATACH

In [12]:
path = "../../sample/ATACH/"
df25 = pd.read_csv(path + "df25.csv")
df50 = pd.read_csv(path + "df50.csv")

data_lst = [df25, df50]

In [13]:
x_cols = ['treatment', 'age', 'ICH volume',
       'ICH Location', 'IVH volume', 'GCS score', 'NIHSS score',
       'Systolic blood pressure',
       'Diastolic Blood Pressure',
       'Hypertension', 'Hyperlipidemia',
       'Type I Diabetes', 'Type II Diabetes',
       'Congestive heart failure', 'Atrial Fibrillation',
       'PTCA',
       'Peripheral Vascular Disease',
       'Myocardial fraction', 'Anti-diabetic',
       'Antihypertensives', 'White blood count', 'Hemoglobin',
       'Hematocrit', 'Platelet count', 'APTT', 'INR', 'Glucose', 'Sodium',
       'Potassium', 'Chloride', 'CD', 'Blood urea nitrogen', 'Creatinine',
       'race', 'sex', 'ethnicity']
y_col = 'mRS score after 30 days'

cols = x_cols + [y_col]
cate_var = ['race', 'ICH Location', 'sex', 'ethnicity']
bool_var = ['treatment','Hypertension', 'Hyperlipidemia',
       'Type I Diabetes', 'Type II Diabetes',
       'Congestive heart failure', 'Atrial Fibrillation',
       'PTCA',
       'Peripheral Vascular Disease',
       'Myocardial fraction', 'Anti-diabetic',
       'Antihypertensives']
cate_cols =[]
cate_ = cate_var + bool_var
for var in cate_:
    cate_cols.append(cols.index(var))

num_var = list((set(cols) - set(cate_var)) - set(bool_var))

In [14]:
ns = [25, 50]
for i in range(2):
    data_len = ns[i]
    data = data_lst[i]
    test_data = pd.read_csv(path + f"df{data_len}_test.csv")
    data = data[cols]
    data_comb = pd.concat([data, test_data])
    data_comb.reset_index(inplace=True, drop=True)
    encoder_dict = {}
    for col in cate_var:
        encoder = LabelEncoder()
        print(col)
        encoder.fit(data_comb[col])
        data[col] = encoder.transform(data[col])
        encoder_dict[col] = encoder
    X = torch.tensor(data.to_numpy())
    clf = TabPFNClassifier()
    reg = TabPFNRegressor()
    model = unsupervised.TabPFNUnsupervisedModel(
        tabpfn_clf = clf, tabpfn_reg = reg
    )

    model.set_categorical_features(cate_cols)
    model.fit(X)
    for j in range(5):
        torch.manual_seed(j)
        torch.cuda.manual_seed(j)
        torch.cuda.manual_seed_all(j)
        np.random.seed(j)
        random.seed(j)
        syn = model.generate_synthetic_data(n_samples=data_len)
        syn_df = pd.DataFrame(syn.numpy(), columns=cols)
        for k, encoder in encoder_dict.items():
            syn_df[k] = syn_df[k].astype(int)
            syn_df[k] = encoder.inverse_transform(syn_df[k])
        syn_df.to_csv(f"gen/ATACH/{data_len}/TabPFN/df_{j}.csv", index=False)

race
ICH Location
sex
ethnicity


  0%|          | 0/37 [00:00<?, ?it/s]

0


  3%|▎         | 1/37 [00:00<00:11,  3.26it/s]

1


...

/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:227: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_sampled = pred["criterion"].sample(torch.tensor(pred["logits"]), t=t)


36


/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:227: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_sampled = pred["criterion"].sample(torch.tensor(pred["logits"]), t=t)


36


/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:227: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_sampled = pred["criterion"].sample(torch.tensor(pred["logits"]), t=t)
/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:166: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(d["logits"]) for d in densities],
100%|██████████| 37/37 [00:45<00:00,  1.24s/it]


In [29]:
df100= pd.read_csv(path + "data100.csv")
df200 = pd.read_csv(path + "data200.csv")

data_lst = [df100, df200]

In [32]:
ns = [100, 200]
for i in range(2):
    data_len = ns[i]
    data = data_lst[i]
    test_data = pd.read_csv(path + f"data_test.csv")
    data = data[cols]
    data_comb = pd.concat([data, test_data])
    data_comb.reset_index(inplace=True, drop=True)
    encoder_dict = {}
    for col in cate_var:
        encoder = LabelEncoder()
        print(col)
        encoder.fit(data_comb[col])
        data[col] = encoder.transform(data[col])
        encoder_dict[col] = encoder
    X = torch.tensor(data.to_numpy())
    clf = TabPFNClassifier()
    reg = TabPFNRegressor()
    model = unsupervised.TabPFNUnsupervisedModel(
        tabpfn_clf = clf, tabpfn_reg = reg
    )

    model.set_categorical_features(cate_cols)
    model.fit(X)
    for j in range(5):
        torch.manual_seed(j)
        torch.cuda.manual_seed(j)
        torch.cuda.manual_seed_all(j)
        np.random.seed(j)
        random.seed(j)
        syn = model.generate_synthetic_data(n_samples=data_len)
        syn_df = pd.DataFrame(syn.numpy(), columns=cols)
        for k, encoder in encoder_dict.items():
            syn_df[k] = syn_df[k].astype(int)
            syn_df[k] = encoder.inverse_transform(syn_df[k])
        syn_df.to_csv(f"gen/ATACH/{data_len}/TabPFN/df_{j}.csv", index=False)

race
ICH Location
sex
ethnicity


  0%|          | 0/37 [00:00<?, ?it/s]

0


  3%|▎         | 1/37 [00:00<00:20,  1.73it/s]

1


...

36


/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:227: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_sampled = pred["criterion"].sample(torch.tensor(pred["logits"]), t=t)


36


/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:227: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_sampled = pred["criterion"].sample(torch.tensor(pred["logits"]), t=t)
/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:166: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(d["logits"]) for d in densities],
100%|██████████| 37/37 [00:43<00:00,  1.17s/it]


OSError: Cannot save file into a non-existent directory: 'gen/ATACH/100/TabPFN'

# Magic

In [20]:
path = "../../sample/Magic/"
ns = [25, 50, 100, 200, 400, 800] 
x_cols = ['major axis of ellipse (mm)', 'minor axis of ellipse (mm)',
       '10-log of sum of content of all pixels (#phot)',
       'ratio of sum of two highest pixels over fSize',
       'ratio of highest pixel over fSize',
       'distance from highest pixel to center, projected onto major axis',
       '3rd root of third moment along major axis (mm)',
       '3rd root of third moment along minor axis (mm)',
       'angle of major axis with vector to origin (deg)',
       'distance from origin to center of ellipse (mm)',]
y_col = 'gamma (signal), hadron (background)'
cate_var = []
bool_var = ['gamma (signal), hadron (background)']
num_var = ['major axis of ellipse (mm)', 'minor axis of ellipse (mm)',
       '10-log of sum of content of all pixels (#phot)',
       'ratio of sum of two highest pixels over fSize',
       'ratio of highest pixel over fSize',
       'distance from highest pixel to center, projected onto major axis',
       '3rd root of third moment along major axis (mm)',
       '3rd root of third moment along minor axis (mm)',
       'angle of major axis with vector to origin (deg)']
cate_cols = bool_var + cate_var
    
cols = x_cols + [y_col]

In [11]:
test_data = pd.read_csv(path + "df_test.csv")
for i in range(len(ns)):
    data_len = ns[i]
    data = pd.read_csv(path + f"df{data_len}.csv")
    data = data[cols]
    data_comb = pd.concat([data, test_data])
    data_comb.reset_index(inplace=True, drop=True)
    encoder_dict = {}
    for col in cate_cols:
        encoder = LabelEncoder()
        print(col)
        encoder.fit(data_comb[col])
        data[col] = encoder.transform(data[col])
        encoder_dict[col] = encoder
    X = torch.tensor(data.to_numpy())
    clf = TabPFNClassifier()
    reg = TabPFNRegressor()
    model = unsupervised.TabPFNUnsupervisedModel(
        tabpfn_clf = clf, tabpfn_reg = reg
    )

    model.set_categorical_features(cate_cols)
    model.fit(X)

    path_tabpfn = f"gen/Magic/{data_len}/TabPFN/"
    os.makedirs(path_tabpfn, exist_ok=True)
    
    for j in range(5):
        torch.manual_seed(j)
        torch.cuda.manual_seed(j)
        torch.cuda.manual_seed_all(j)
        np.random.seed(j)
        random.seed(j)
        syn = model.generate_synthetic_data(n_samples=data_len)
        syn_df = pd.DataFrame(syn.numpy(), columns=cols)
        
        for k, encoder in encoder_dict.items():
            syn_df[k] = syn_df[k].astype(int)
            syn_df[k] = encoder.inverse_transform(syn_df[k])
        syn_df.to_csv(f"gen/Magic/{data_len}/TabPFN/df_{j}.csv", index=False)

gamma (signal), hadron (background)


  0%|          | 0/11 [00:00<?, ?it/s]

0


/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:227: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_sampled = pred["criterion"].sample(torch.tensor(pred["logits"]), t=t)
/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:166: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(d["logits"]) for d in densities],
  9%|▉         | 1/11 [00:00<00:04,  2.31it/s]

1


...

/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:227: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_sampled = pred["criterion"].sample(torch.tensor(pred["logits"]), t=t)


10


/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:227: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_sampled = pred["criterion"].sample(torch.tensor(pred["logits"]), t=t)
/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:166: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(d["logits"]) for d in densities],
100%|██████████| 11/11 [00:14<00:00,  1.30s/it]


gamma (signal), hadron (background)
10


KeyError: 10

In [21]:
test_data = pd.read_csv(path + "df_test.csv")
ns = [400, 800] 

for i in range(len(ns)):
    data_len = ns[i]
    data = pd.read_csv(path + f"df{data_len}.csv")
    data = data[cols]
    data_comb = pd.concat([data, test_data])
    data_comb.reset_index(inplace=True, drop=True)
    encoder_dict = {}
    for col in cate_cols:
        encoder = LabelEncoder()
        print(col)
        encoder.fit(data_comb[col])
        data[col] = encoder.transform(data[col])
        encoder_dict[col] = encoder
    X = torch.tensor(data.to_numpy())
    clf = TabPFNClassifier()
    reg = TabPFNRegressor()
    model = unsupervised.TabPFNUnsupervisedModel(
        tabpfn_clf = clf, tabpfn_reg = reg
    )

    model.set_categorical_features(cate_cols)
    model.fit(X)

    path_tabpfn = f"gen/Magic/{data_len}/TabPFN/"
    os.makedirs(path_tabpfn, exist_ok=True)
    
    for j in range(5):
        torch.manual_seed(j)
        torch.cuda.manual_seed(j)
        torch.cuda.manual_seed_all(j)
        np.random.seed(j)
        random.seed(j)
        syn = model.generate_synthetic_data(n_samples=data_len)
        syn_df = pd.DataFrame(syn.numpy(), columns=cols)
        
        for k, encoder in encoder_dict.items():
            syn_df[k] = syn_df[k].astype(int)
            syn_df[k] = encoder.inverse_transform(syn_df[k])
        syn_df.to_csv(f"gen/Magic/{data_len}/TabPFN/df_{j}.csv", index=False)

gamma (signal), hadron (background)


  0%|          | 0/11 [00:00<?, ?it/s]

0


/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:227: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_sampled = pred["criterion"].sample(torch.tensor(pred["logits"]), t=t)
/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:166: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(d["logits"]) for d in densities],
  9%|▉         | 1/11 [00:00<00:06,  1.46it/s]

1


...

/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:227: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_sampled = pred["criterion"].sample(torch.tensor(pred["logits"]), t=t)


10


/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:227: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_sampled = pred["criterion"].sample(torch.tensor(pred["logits"]), t=t)
/home/Paper benchmark/Paper revision/baselines/tabpfn-community/src/tabpfn_extensions/unsupervised/unsupervised.py:166: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(d["logits"]) for d in densities],
100%|██████████| 11/11 [00:15<00:00,  1.44s/it]


gamma (signal), hadron (background)
10


KeyError: 10